# snap2vec

usinn stanford social network datasets + word2vec

In [1]:
# you'll probably get a lot of warnings here -- sorry 
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import json
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
from sklearn.mixture import GaussianMixture as GMM
from sklearn.decomposition import PCA
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import hdbscan
% matplotlib inline
output_notebook()

/Users/gmerz/person2vec/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


Loading BokehJS ...

## `1. Preprocessing 

We want to write both sparsified and regular forms of our graph to adjaceny list. We want to test word2vec on various sparsified graphs and see if we can predict links that occur in the full on adjacency list

*Write Full graph*

In [2]:
! python preprocess_snaps.py twitter_combined.txt 1 directed

Constructing adjacency list
Writing to file


*Write 90% of edges*

In [3]:
! python preprocess_snaps.py twitter_combined.txt .9 directed

Constructing adjacency list
Writing to file


*Write 80% of edges*

In [4]:
! python preprocess_snaps.py twitter_combined.txt .8 directed

Constructing adjacency list
Writing to file


*Write 70% of edges*

In [5]:
! python preprocess_snaps.py twitter_combined.txt .7 directed

Constructing adjacency list
Writing to file


In [6]:
def adjListFromFile(fpath):
    ''' 
        function that constructs a dictionary 
        of nodes -> list of nodes their connected to
        (an adjacency list representation of the graph)
        from a file
    '''
    adjList = {}
    with open(fpath, 'r') as f: 
        for i, line in enumerate(f): 
            if i != 0: 
                totalList = line.strip().split(' ')
                adjList[totalList[0]] = [x.strip() for x in totalList[1:]]
    return adjList 

In [7]:
fullGraph = adjListFromFile('twitter_combine1_lst.txt')
partialGraph = adjListFromFile('twitter_combine0.9_lst.txt')
partialGraphS = adjListFromFile('twitter_combine0.7_lst.txt')

In [8]:
# sanity check 
print(len(fullGraph), len(partialGraphS), len(partialGraph))

70097 67930 69499


In [9]:
def testNodes(partialGraph, fullGraph):
    '''
        another quick sanity check: function that 
        ensures every node in the partial graph
        is also in the full graph 
        
        (this wasn't the case before -- there was a bug
        where spaces weren't written but i fixed hehe)
    '''
    num_errors = 0 
    for node_id in partialGraph.keys(): 
        if node_id not in fullGraph.keys(): 
            num_errors += 1 
    return num_errors

assert not(testNodes(partialGraph, fullGraph))
assert not(testNodes(partialGraphS, fullGraph))

In [10]:
! python documentize.py twitter_combine0.9_lst.txt twitter.9vec.txt .01 100000

PROCESSING GRAPH
WRITING TO FILE


In [11]:
! python documentize.py twitter_combine0.7_lst.txt twitter.7vec.txt .01 100000

PROCESSING GRAPH
WRITING TO FILE


## 2. word2vec

In [12]:
%%time
model = Word2Vec(LineSentence('twitter.7vec.txt'), size=100, window=5, min_count=10, workers=4)

CPU times: user 15.9 s, sys: 304 ms, total: 16.2 s
Wall time: 9.24 s


In [14]:
def testPrediction(fullGraph, partialGraph, model):
    numExisting, numFound, numWrong = 0, 0, 0
    for i, user_id in enumerate(model.vocab.keys()):
        if i % 20000 == 0: 
            print("On word %s out of %s" % (i, len(model.vocab)))
        most_similar = [t[0] for t in model.most_similar(user_id)[:5]]
        for similar_user in most_similar: 
            if similar_user in partialGraph[user_id]:
                numExisting += 1
            elif similar_user in fullGraph[user_id]: 
                numFound += 1 
            else: 
                numWrong += 1 
    return numExisting, numFound, numWrong

In [15]:
testPrediction(fullGraph, partialGraphS, model)

On word 0 out of 16557


(17914, 3533, 61338)

# 3. Comparison with LinkPred

In [17]:
import linkpred

In [18]:
# copy of the file just renamed .edgelist so it knows what to do lmaooo
G = linkpred.read_network("twitter_combined.edgelist")

In [ ]:
simrank = linkpred.predictors.SimRank(G, excluded=G.edges())

In [ ]:
simrank_results = simrank.predict(c=0.5)